In [ ]:
# pip install scikit-surprise

Note: you may need to restart the kernel to use updated packages.Collecting scikit-surprise
  Using cached scikit_surprise-1.1.4.tar.gz (154 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp312-cp312-win_amd64.whl size=1291216 sha256=ca4456063ee2a834c621d6e5b3f4f4622adca521d5e59011c8304f245043dda2
  Stored in directory: c:\users\lenovo\appdata\local\pip\cache\wheels\75\fa\bc\739bc2cb1fbaab6061854e6cfbb81a0ae52c92a502a7fa454b
Successfully built scikit-surprise



  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from surprise import Dataset, Reader
from surprise import KNNBasic
from surprise.model_selection import cross_validate

In [3]:
ratings_df = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1/main/Datasets/ratings.csv')
movies_df = pd.read_csv('https://raw.githubusercontent.com/delinai/schulich_ds1/main/Datasets/movies.csv')

In [4]:
data = pd.merge(ratings_df, movies_df, on='movieId')

In [5]:
reader = Reader(rating_scale=(1,10))
data = Dataset.load_from_df(data[['userId','movieId','rating']], reader)

In [6]:
sim_options = {'name':'cosine',
               'user_based': False} # content based filtering

In [7]:
algo = KNNBasic(sim_options=sim_options)

In [8]:
cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=True)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9704  0.9783  0.9736  0.9746  0.9713  0.9737  0.0028  
Fit time          239.48  63.39   21.78   22.82   16.33   72.76   85.04   
Test time         31.82   10.17   10.32   21.67   9.92    16.78   8.75    


{'test_rmse': array([0.97042822, 0.97828084, 0.9736172 , 0.97463891, 0.97130288]),
 'fit_time': (239.47594380378723,
  63.38895535469055,
  21.775115966796875,
  22.821747303009033,
  16.32533574104309),
 'test_time': (31.817626476287842,
  10.165523290634155,
  10.315550088882446,
  21.666220664978027,
  9.92394232749939)}

In [9]:
algo.fit(data.build_full_trainset())

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [10]:
print(algo.sim)

[[1.         0.90773108 0.96657344 ... 1.         1.         1.        ]
 [0.90773108 1.         0.88581521 ... 1.         1.         1.        ]
 [0.96657344 0.88581521 1.         ... 1.         1.         1.        ]
 ...
 [1.         1.         1.         ... 1.         1.         1.        ]
 [1.         1.         1.         ... 1.         1.         1.        ]
 [1.         1.         1.         ... 1.         1.         1.        ]]


In [11]:
pred = algo.predict(uid=1, iid=10)
pred.est

3.9880309408033616

In [12]:
df = pd.merge(ratings_df, movies_df, on='movieId')

In [13]:
sim_options = {'name':'cosine',
               'user_based': False,
               'k':10} # content based filtering

In [14]:
algo = KNNBasic(sim_options=sim_options)

In [15]:
similarities = algo.sim

AttributeError: 'KNNBasic' object has no attribute 'sim'

In [ ]:
sns.heatmap(np.array(similarities), annot=True)

In [ ]:
from surprise.model_selection import train_test_split
from surprise import accuracy

In [ ]:
trainset, testset = train_test_split(data, test_size=0.25)

In [ ]:
algo = KNNBasic(sim_options={'user_based':True})

In [ ]:
algo.fit(trainset)

In [ ]:
predictions = algo.test(testset)

In [ ]:
# Using the same initial setup as in the user-based example

# Using KNN to compute similarities between items
algo = KNNBasic(sim_options={'user_based': False})
algo.fit(trainset)

# Making predictions and evaluating the model
predictions = algo.test(testset)
accuracy.rmse(predictions)

In [ ]:
predictions

In [ ]:
df[df['userId']==1]['movieId'].unique()

In [ ]:
# SVD
from surprise import SVD

In [ ]:
svd = SVD()
cross_validate(svd, data, measures=['RMSE'], cv=5, verbose=True)

In [ ]:
trainset2 = data.build_full_trainset()
svd.fit(trainset)

In [ ]:
svd.predict(1,10)

In [ ]:
# predictions for everyone
df['predicted_rating'] = df.apply(lambda x: svd.predict(x['userId'], x['movieId']).est, axis=1)
df